# Advanced Retrieval with LangChain

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

# 🤝 Breakout Room Part #1

In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [2]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

In [3]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

loader = CSVLoader(
    file_path=f"./data/Projects_with_Domains.csv",
    metadata_columns=[
      "Project Title",
      "Project Domain",
      "Secondary Domain",
      "Description",
      "Judge Comments",
      "Score",
      "Project Name",
      "Judge Score"
    ]
)

synthetic_usecase_data = loader.load()

for doc in synthetic_usecase_data:
    doc.page_content = doc.metadata["Description"]

In [4]:
synthetic_usecase_data[0]

Document(metadata={'source': './data/Projects_with_Domains.csv', 'row': 0, 'Project Title': 'InsightAI 1', 'Project Domain': 'Security', 'Secondary Domain': 'Finance / FinTech', 'Description': 'A low-latency inference system for multimodal agents in autonomous systems.', 'Judge Comments': 'Technically ambitious and well-executed.', 'Score': '85', 'Project Name': 'Project Aurora', 'Judge Score': '9.5'}, page_content='A low-latency inference system for multimodal agents in autonomous systems.')

In [5]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    synthetic_usecase_data,
    embeddings,
    location=":memory:",
    collection_name="Synthetic_Usecases"
)

## Task 4: Naive RAG Chain

In [6]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

In [7]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

In [8]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4.1-nano")

### LCEL RAG Chain

In [9]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [10]:
naive_retrieval_chain.invoke({"question" : "What is the most common project domain?"})["response"].content

'Based on the provided data, the most common project domain appears to be "Healthcare / MedTech," which is mentioned multiple times in different entries.'

In [11]:
naive_retrieval_chain.invoke({"question" : "Were there any usecases about security?"})["response"].content

'Yes, there are usecases related to security. Specifically, one project titled "TrendLens 19" describes a federated learning toolkit that improves privacy in healthcare applications, which is related to data security and privacy.'

In [12]:
naive_retrieval_chain.invoke({"question" : "What did judges have to say about the fintech projects?"})["response"].content

'The judges had generally positive comments about the fintech projects. They described some projects as "a clever solution with measurable environmental benefit," "technically ambitious and well-executed," and noted "promising ideas with robust experimental validation." Additionally, comments such as "solid work with impressive real-world impact" and "excellent code quality and use of open-source libraries" were made. Overall, the judges recognized the projects\' technical strengths, innovative approaches, and potential real-world benefits.'

## Task 5: Best-Matching 25 (BM25) Retriever

In [13]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(synthetic_usecase_data)

In [14]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [15]:
bm25_retrieval_chain.invoke({"question" : "What is the most common project domain?"})["response"].content

'The most common project domain cannot be determined definitively from the provided data snippet, as it only shows a few projects. However, among the sample projects listed, the following domains are mentioned:\n\n- Productivity Assistants\n- E‑commerce / Marketplaces\n- Healthcare / MedTech\n- Finance / FinTech\n\nSince "Finance / FinTech" appears twice in this small sample, it might be a common domain, but I do not have enough data to conclusively identify the most common project domain overall.'

In [16]:
bm25_retrieval_chain.invoke({"question" : "Were there any usecases about security?"})["response"].content

'Yes, there is a use case related to security. The project titled "SecureNest 49" falls under the domain of E‑commerce / Marketplaces with a secondary focus on Legal / Compliance. It involves a document summarization and retrieval system for enterprise knowledge bases, which suggests it addresses security and compliance considerations in managing sensitive information.'

In [17]:
bm25_retrieval_chain.invoke({"question" : "What did judges have to say about the fintech projects?"})["response"].content

'The judges had a positive view of the fintech projects. Specifically, for the project "SynthMind," which is in the finance/fintech domain, the judge comments were that it is "Conceptually strong but results need more benchmarking," and it received a high judge score of 9.6.'

#### ❓ Question #1:

Give an example query where BM25 is better than embeddings and justify your answer.

##### ✅ Answer


## Task 6: Contextual Compression (Using Reranking)

In [18]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-v3.5")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [19]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [20]:
contextual_compression_retrieval_chain.invoke({"question" : "What is the most common project domain?"})["response"].content

'Based on the provided data, the project domains mentioned include Healthcare / MedTech, Security, and Creative / Design / Media. Since only a few entries are shown, and Healthcare / MedTech appears multiple times across the sample, it suggests that Healthcare / MedTech may be a common project domain. However, with only this limited data, I cannot definitively determine the most common project domain.'

In [21]:
contextual_compression_retrieval_chain.invoke({"question" : "Were there any usecases about security?"})["response"].content

'Based on the provided context, there are no specific use cases related to security mentioned. The use cases focus on federated learning to improve privacy in healthcare applications, but security is not explicitly referenced.'

In [22]:
contextual_compression_retrieval_chain.invoke({"question" : "What did judges have to say about the fintech projects?"})["response"].content

'The judges had positive comments about the fintech project "PlanPilot 35" (also known as "SkyForge"). They described it as "a clever solution with measurable environmental benefit," indicating that the judges appreciated the innovative approach and its potential impact.'

## Task 7: Multi-Query Retriever

In [23]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
) 

In [24]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [25]:
multi_query_retrieval_chain.invoke({"question" : "What is the most common project domain?"})["response"].content

'Based on the data provided, the most common project domain appears to be "Finance / FinTech," which is mentioned multiple times in the dataset.'

In [26]:
multi_query_retrieval_chain.invoke({"question" : "Were there any usecases about security?"})["response"].content

'Yes, there are use cases related to security. Specifically, one project titled "Project Aurora" focuses on a low-latency inference system for multimodal agents in autonomous systems, which falls under the security domain.'

In [27]:
multi_query_retrieval_chain.invoke({"question" : "What did judges have to say about the fintech projects?"})["response"].content

'Judges generally provided positive feedback on the fintech-related projects. For example, one project, "SecureNest," was described as having a strong conceptual foundation, although it needed more benchmarking of results. Another fintech project, "Pathfinder 27," was praised for its excellent code quality and use of open-source libraries. Overall, judges recognized the projects as promising, with comments highlighting their potential impact and strong technical execution.'

#### ❓ Question #2:

Explain how generating multiple reformulations of a user query can improve recall.

##### ✅ Answer


## Task 8: Parent Document Retriever

In [28]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = synthetic_usecase_data
child_splitter = RecursiveCharacterTextSplitter(chunk_size=750)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [29]:
from langchain_qdrant import QdrantVectorStore

client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = QdrantVectorStore(
    collection_name="full_documents", embedding=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

In [30]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

In [31]:
parent_document_retriever.add_documents(parent_docs, ids=None)

In [32]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [33]:
parent_document_retrieval_chain.invoke({"question" : "What is the most common project domain?"})["response"].content

'Based on the provided data, the most common project domain appears to be "Healthcare / MedTech," as it is mentioned multiple times in the sample. However, since the data provided is limited to a few entries, I cannot definitively determine the most common project domain overall. \n\nIf you have access to the full dataset, I recommend analyzing the full list to find the domain with the highest frequency.'

In [34]:
parent_document_retrieval_chain.invoke({"question" : "Were there any usecases about security?"})["response"].content

'Based on the provided context, there are no specific use cases explicitly related to security. The projects mentioned focus on federated learning to improve privacy in healthcare applications, which is related to security, but there are no direct references to security use cases.'

In [35]:
parent_document_retrieval_chain.invoke({"question" : "What did judges have to say about the fintech projects?"})["response"].content

'The judges made the following remarks about the fintech projects:\n\n- In the case of "WealthifyAI," they described it as having a "comprehensive and technically mature approach."'

## Task 9: Ensemble Retriever

In [36]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

In [37]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [38]:
ensemble_retrieval_chain.invoke({"question" : "What is the most common project domain?"})["response"].content

'Based on the provided data, the most common project domain appears to be "Healthcare / MedTech," which is mentioned multiple times in the dataset.'

In [39]:
ensemble_retrieval_chain.invoke({"question" : "Were there any usecases about security?"})["response"].content

'Yes, there is at least one use case related to security. Specifically, the project titled "SecureNest 49" involves a document summarization and retrieval system for enterprise knowledge bases. Additionally, "SecureNest 28" is a hardware-aware model quantization benchmark suite that could relate to security in hardware or model robustness.'

In [40]:
ensemble_retrieval_chain.invoke({"question" : "What did judges have to say about the fintech projects?"})["response"].content

'Judges had varied comments about the fintech projects. For example, they described the project "Pathfinder 27" as having excellent code quality and effective use of open-source libraries, giving it a high score of 9.8. They also appreciated the fintech project "SynthMind," noting that it had a strong conceptual foundation but needed more benchmarking results, and awarded it a high score of 9.6. Overall, judges recognized the strengths of these fintech projects, highlighting qualities such as good quality, innovative ideas, and technical strength.'

## Task 10: Semantic Chunking

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [41]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

In [42]:
semantic_documents = semantic_chunker.split_documents(synthetic_usecase_data[:20])

In [43]:
semantic_vectorstore = Qdrant.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="Synthetic_Usecase_Data_Semantic_Chunks"
)

In [44]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

In [45]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [46]:
semantic_retrieval_chain.invoke({"question" : "What is the most common project domain?"})["response"].content

'Based on the provided data, the most common project domain is "Legal / Compliance," which appears twice in the sample provided. However, since the complete dataset isn\'t visible and only a small subset is shown, I cannot definitively determine which domain is most common overall. \n\nIf you are referring to just this snippet, then "Legal / Compliance" is the most common. If you have access to the full dataset and need an exact answer, I recommend analyzing the entire list to identify the most frequently appearing domain.'

In [47]:
semantic_retrieval_chain.invoke({"question" : "Were there any usecases about security?"})["response"].content

'Yes, there are use cases related to security in the provided information. Specifically, the projects "MediMind" (BioForge) and "InsightAI" (Project Aurora) are categorized under the domain of Security.'

In [48]:
semantic_retrieval_chain.invoke({"question" : "What did judges have to say about the fintech projects?"})["response"].content

'Judges generally had positive comments about the fintech projects. For example, one judge described "AutoMate" as a "forward-looking idea with solid supporting data," and another noted "WealthifyAI 16" as having a "comprehensive and technically mature approach." Additionally, "InsightAI 1" was praised as "technically ambitious and well-executed" with a high score of 9.5. Overall, the judges highlighted qualities such as technical ambition, maturity, and solid supporting data in their remarks on the fintech projects.'

#### ❓ Question #3:

If sentences are short and highly repetitive (e.g., FAQs), how might semantic chunking behave, and how would you adjust the algorithm?

##### ✅ Answer


# 🤝 Breakout Room Part #2

#### 🏗️ Activity #1

Your task is to evaluate the various Retriever methods against eachother.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparision between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

In [49]:
### YOUR CODE HERE